### Konstantina Andronikou

## Transitive Vs Intransitive Verbs 

In [25]:
from allennlp_models.pretrained import load_predictor

In [26]:
import checklist
import logging 
import csv
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

In [27]:
from checklist.pred_wrapper import PredictorWrapper

## Intransitive 

In [28]:
# remove warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True
srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Carla pays")
output

{'verbs': [{'verb': 'pays',
   'description': '[ARG0: Carla] [V: pays]',
   'tags': ['B-ARG0', 'B-V']}],
 'words': ['Carla', 'pays']}

In [29]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [30]:
d = ["Carla pays"]
pred = predict_and_conf(d)
pred

([{'verbs': [{'verb': 'pays',
     'description': '[ARG0: Carla] [V: pays]',
     'tags': ['B-ARG0', 'B-V']}],
   'words': ['Carla', 'pays']}],
 array([1.]))

In [31]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [33]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [34]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [35]:
def found_arg0_people(x, pred, conf, label=None, meta=None):

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [36]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays', meta=True, nsamples=1000, remove_duplicates = True)

for k, v in t.items():
    print(k,v)

meta [{'first_name': 'Frank'}, {'first_name': 'Colin'}, {'first_name': 'Kevin'}, {'first_name': 'Nancy'}, {'first_name': 'Leslie'}, {'first_name': 'Ellen'}, {'first_name': 'Billy'}, {'first_name': 'Charlotte'}, {'first_name': 'Scott'}, {'first_name': 'Sharon'}, {'first_name': 'Rachel'}, {'first_name': 'Robert'}, {'first_name': 'Louise'}, {'first_name': 'Peter'}, {'first_name': 'Robert'}, {'first_name': 'Sara'}, {'first_name': 'Kathleen'}, {'first_name': 'Greg'}, {'first_name': 'Ralph'}, {'first_name': 'Harriet'}, {'first_name': 'Anna'}, {'first_name': 'Elizabeth'}, {'first_name': 'Bill'}, {'first_name': 'Susan'}, {'first_name': 'Paul'}, {'first_name': 'Samuel'}, {'first_name': 'Ian'}, {'first_name': 'Lynn'}, {'first_name': 'Sara'}, {'first_name': 'Kathy'}, {'first_name': 'Annie'}, {'first_name': 'Jill'}, {'first_name': 'Michael'}, {'first_name': 'Frank'}, {'first_name': 'Deborah'}, {'first_name': 'Steve'}, {'first_name': 'Tom'}, {'first_name': 'Gary'}, {'first_name': 'Kathryn'}, {'firs

In [37]:
with open('dataset/Intransitive.txt', 'w') as f:
    print(t.data, file = f)

In [38]:
test = MFT(**t, name = 'detect_arg0name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Intransitive.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    12 (1.2%)

Example fails:
[ARG1: Alexandra] [V: pays]
----
[ARG1: Lawrence] [V: pays]
----
[ARG1: Lawrence] [V: pays]
----
['Frank', 'pays'] [ True]
['Colin', 'pays'] [ True]
['Kevin', 'pays'] [ True]
['Nancy', 'pays'] [ True]
['Leslie', 'pays'] [ True]
['Ellen', 'pays'] [ True]
['Billy', 'pays'] [ True]
['Charlotte', 'pays'] [ True]
['Scott', 'pays'] [ True]
['Sharon', 'pays'] [ True]
['Rachel', 'pays'] [ True]
['Robert', 'pays'] [ True]
['Louise', 'pays'] [ True]
['Peter', 'pays'] [ True]
['Robert', 'pays'] [ True]
['Sara', 'pays'] [ True]
['Kathleen', 'pays'] [ True]
['Greg', 'pays'] [ True]
['Ralph', 'pays'] [ True]
['Harriet', 'pays'] [ True]
['Anna', 'pays'] [ True]
['Elizabeth', 'pays'] [ True]
['Bill', 'pays'] [ True]
['Susan', 'pays'] [ True]
['Paul', 'pays'] [ True]
['Samuel', 'pays'] [ True]
['Ian', 'pays'] [ True]
['Lynn', 'pays'] [ True]
['Sara', 'pays'] [ True]
['Kathy', 'pays'] [ True]
['Annie', 'pays'] [ Tr

In [39]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Intransitive.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive 

In [40]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [41]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [42]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [43]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [44]:
editor = Editor()

# create examples
t = editor.template('{first_name} pays for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Stephanie'}, {'first_name': 'Tom'}, {'first_name': 'Francis'}, {'first_name': 'Daniel'}, {'first_name': 'Kathy'}, {'first_name': 'Donald'}, {'first_name': 'Rebecca'}, {'first_name': 'Patrick'}, {'first_name': 'Michelle'}, {'first_name': 'Anne'}, {'first_name': 'Daniel'}, {'first_name': 'Sarah'}, {'first_name': 'Florence'}, {'first_name': 'Victoria'}, {'first_name': 'Katherine'}, {'first_name': 'Tony'}, {'first_name': 'Dorothy'}, {'first_name': 'Anna'}, {'first_name': 'Grace'}, {'first_name': 'Benjamin'}, {'first_name': 'Jerry'}, {'first_name': 'Andrew'}, {'first_name': 'Christopher'}, {'first_name': 'Leslie'}, {'first_name': 'Helen'}, {'first_name': 'Carl'}, {'first_name': 'Caroline'}, {'first_name': 'Diana'}, {'first_name': 'Eric'}, {'first_name': 'Karen'}, {'first_name': 'Louis'}, {'first_name': 'Greg'}, {'first_name': 'Ron'}, {'first_name': 'Richard'}, {'first_name': 'Daniel'}, {'first_name': 'Harriet'}, {'first_name': 'Mark'}, {'first_name': 'Mary'}, {'first_n

In [45]:
with open('dataset/Transitive.txt', 'w') as f:
    print(t.data, file = f)

In [46]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Transitive.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    22 (2.2%)

Example fails:
[ARG1: Sally] [V: pays] [ARG3: for coffee] .
----
[ARG1: Margaret] [V: pays] [ARG3: for coffee] .
----
[ARG1: Alison] [V: pays] [ARG3: for coffee] .
----
['Stephanie', 'pays', 'for', 'coffee', '.'] [ True]
['Tom', 'pays', 'for', 'coffee', '.'] [ True]
['Francis', 'pays', 'for', 'coffee', '.'] [ True]
['Daniel', 'pays', 'for', 'coffee', '.'] [ True]
['Kathy', 'pays', 'for', 'coffee', '.'] [ True]
['Donald', 'pays', 'for', 'coffee', '.'] [ True]
['Rebecca', 'pays', 'for', 'coffee', '.'] [ True]
['Patrick', 'pays', 'for', 'coffee', '.'] [ True]
['Michelle', 'pays', 'for', 'coffee', '.'] [ True]
['Anne', 'pays', 'for', 'coffee', '.'] [ True]
['Daniel', 'pays', 'for', 'coffee', '.'] [ True]
['Sarah', 'pays', 'for', 'coffee', '.'] [ True]
['Florence', 'pays', 'for', 'coffee', '.'] [ True]
['Victoria', 'pays', 'for', 'coffee', '.'] [ True]
['Katherine', 'pays', 'for', 'coffee', '.'] [ True]
['Tony', 'pay

In [47]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Transitive.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive-Intransitive Robustness Test

In [52]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Eleanor'}, {'first_name': 'Joan'}, {'first_name': 'Catherine'}, {'first_name': 'George'}, {'first_name': 'Julia'}, {'first_name': 'Ben'}, {'first_name': 'Mark'}, {'first_name': 'Matthew'}, {'first_name': 'Annie'}, {'first_name': 'Nicole'}, {'first_name': 'Diane'}, {'first_name': 'Chris'}, {'first_name': 'Louise'}, {'first_name': 'George'}, {'first_name': 'Stephen'}, {'first_name': 'Sue'}, {'first_name': 'Grace'}, {'first_name': 'Gary'}, {'first_name': 'Lucy'}, {'first_name': 'Don'}, {'first_name': 'Charles'}, {'first_name': 'Benjamin'}, {'first_name': 'Tim'}, {'first_name': 'Lucy'}, {'first_name': 'Katie'}, {'first_name': 'Harriet'}, {'first_name': 'Paul'}, {'first_name': 'Kenneth'}, {'first_name': 'Tim'}, {'first_name': 'Maria'}, {'first_name': 'Kenneth'}, {'first_name': 'Suzanne'}, {'first_name': 'Ann'}, {'first_name': 'Jean'}, {'first_name': 'Emily'}, {'first_name': 'Carl'}, {'first_name': 'Ruth'}, {'first_name': 'Kate'}, {'first_name': 'Evelyn'}, {'first_name'

In [53]:
with open('dataset/Intransitive_Rob.txt', 'w') as f:
    print(t.data, file = f)

In [54]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Intransitive_Rob.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    359 (35.9%)

Example fails:
Henry [V: payed] .
----
Kate [V: payed] .
----
[ARGM-TMP: Anna] [V: payed] .
----
['Eleanor', 'payed', '.'] [ True]
['Joan', 'payed', '.'] [ True]
['Catherine', 'payed', '.'] [ True]
['George', 'payed', '.'] [False]
['Julia', 'payed', '.'] [ True]
['Ben', 'payed', '.'] [False]
['Mark', 'payed', '.'] [False]
['Matthew', 'payed', '.'] [ True]
['Annie', 'payed', '.'] [ True]
['Nicole', 'payed', '.'] [False]
['Diane', 'payed', '.'] [False]
['Chris', 'payed', '.'] [ True]
['Louise', 'payed', '.'] [False]
['George', 'payed', '.'] [False]
['Stephen', 'payed', '.'] [ True]
['Sue', 'payed', '.'] [ True]
['Grace', 'payed', '.'] [False]
['Gary', 'payed', '.'] [ True]
['Lucy', 'payed', '.'] [ True]
['Don', 'payed', '.'] [ True]
['Charles', 'payed', '.'] [ True]
['Benjamin', 'payed', '.'] [ True]
['Tim', 'payed', '.'] [False]
['Lucy', 'payed', '.'] [ True]
['Katie', 'payed', '.'] [ True]
['Harriet', 'payed',

In [58]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Intransitive_Rob.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## Transitive

In [55]:
editor = Editor()

# create examples
t = editor.template('{first_name} payed for coffee.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

for k, v in t.items():
    print(k, v)

meta [{'first_name': 'Rebecca'}, {'first_name': 'Jean'}, {'first_name': 'Sally'}, {'first_name': 'Maria'}, {'first_name': 'Howard'}, {'first_name': 'Alan'}, {'first_name': 'Rachel'}, {'first_name': 'Sally'}, {'first_name': 'Donna'}, {'first_name': 'Andrea'}, {'first_name': 'Steven'}, {'first_name': 'Ron'}, {'first_name': 'Stephen'}, {'first_name': 'Rose'}, {'first_name': 'Karen'}, {'first_name': 'Betty'}, {'first_name': 'Susan'}, {'first_name': 'Matthew'}, {'first_name': 'Harold'}, {'first_name': 'Betty'}, {'first_name': 'Alexandra'}, {'first_name': 'Louis'}, {'first_name': 'Martha'}, {'first_name': 'Henry'}, {'first_name': 'Colin'}, {'first_name': 'Philip'}, {'first_name': 'Betty'}, {'first_name': 'Kelly'}, {'first_name': 'Brian'}, {'first_name': 'Gary'}, {'first_name': 'Sam'}, {'first_name': 'Kevin'}, {'first_name': 'Alexandra'}, {'first_name': 'Carol'}, {'first_name': 'Rebecca'}, {'first_name': 'Mary'}, {'first_name': 'Mary'}, {'first_name': 'Adam'}, {'first_name': 'Katie'}, {'first

In [56]:
with open('dataset/Transitive_Rob.txt', 'w') as f:
    print(t.data, file = f)

In [57]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output/Transitive_Rob.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    501 (50.1%)

Example fails:
Tony [V: payed] [ARG1: for coffee] .
----
[ARGM-MNR: Jerry] [V: payed] [ARG1: for coffee] .
----
[ARGM-DIS: James] [V: payed] [ARG1: for coffee] .
----
['Rebecca', 'payed', 'for', 'coffee', '.'] [ True]
['Jean', 'payed', 'for', 'coffee', '.'] [False]
['Sally', 'payed', 'for', 'coffee', '.'] [False]
['Maria', 'payed', 'for', 'coffee', '.'] [False]
['Howard', 'payed', 'for', 'coffee', '.'] [False]
['Alan', 'payed', 'for', 'coffee', '.'] [ True]
['Rachel', 'payed', 'for', 'coffee', '.'] [ True]
['Sally', 'payed', 'for', 'coffee', '.'] [False]
['Donna', 'payed', 'for', 'coffee', '.'] [ True]
['Andrea', 'payed', 'for', 'coffee', '.'] [False]
['Steven', 'payed', 'for', 'coffee', '.'] [ True]
['Ron', 'payed', 'for', 'coffee', '.'] [ True]
['Stephen', 'payed', 'for', 'coffee', '.'] [ True]
['Rose', 'payed', 'for', 'coffee', '.'] [False]
['Karen', 'payed', 'for', 'coffee', '.'] [ True]
['Betty', 'payed',

In [59]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
original_stdout = sys.stdout  
 
with open('Output/false_sentences_Transitive_Rob.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 